# XMLAgent 示例

这个笔记本展示了如何使用LangChain的XMLAgent创建一个天气查询助手。

## 主要功能
- 使用自定义工具(GetWeather)获取城市天气信息
- 利用XML格式的Agent提示模板处理用户查询
- 通过心知天气API获取实时天气数据

## 使用方法
1. 确保设置了必要的环境变量(API_KEY, API_BASEURL)
2. 运行代码即可通过输入城市名查询天气状况
3. 可以根据需要扩展其他工具功能

这个示例展示了如何将外部API与LLM结合，构建实用的应用程序。

In [2]:
import requests
import os
from langchain.tools import BaseTool
from typing import Any,ClassVar
from langchain import hub
from langchain.agents import AgentExecutor,create_xml_agent
from langchain_openai import ChatOpenAI

# 定义工具——查询天气
class GetWeather(BaseTool):
    """
    查询天气API
    name: 工具名称
    description:工具描述    
    """
    name : ClassVar[str] = "GetWeather"
    description : ClassVar[str] = "根据城市名称获取天气数据"
    
    def __init__(self):
        super().__init__()
    
    def get_weather(self,location):
        api_key = "SqRLyH57b4IUtiH1W"
        url = f"https://api.seniverse.com/v3/weather/now.json?key={api_key}&location={location}&language=zh-Hans&unit=c"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            print(f"API返回的结果：{data}")
            # 这里定义返回结构
            weather = {
                "text" : data["results"][0]["now"]["text"],
                "temperature" :data["results"][0]["now"]["temperature"]
            }
            return weather
        else:
            raise Exception(f"天气API调用失败：{response.status_code}")
    
    def _run(self, location):
        return self.get_weather(location=location)
        
weatherTool = GetWeather()

#weatherTool.run({"location" : "深圳"})

# 定义llm
llm = ChatOpenAI(
    model= "qwen-turbo",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL"),
)

# 从hub获取提示词
prompt = hub.pull("hwchase17/xml-agent-convo")

# 定义工具列表
tools = [weatherTool]

# 创建智能体
agent = create_xml_agent(
    llm = llm,
    tools = tools,
    prompt = prompt
)

# 执行智能体
agentExecutor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agentExecutor.invoke({"input" : "深圳的天气怎么样？"})



d:\Anaconda\Lib\site-packages\langsmith\client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
<tool>GetWeather</tool><tool_input>深圳API返回的结果：{'results': [{'location': {'id': 'WS10730EM8EV', 'name': '深圳', 'country': 'CN', 'path': '深圳,深圳,广东,中国', 'timezone': 'Asia/Shanghai', 'timezone_offset': '+08:00'}, 'now': {'text': '多云', 'code': '4', 'temperature': '25'}, 'last_update': '2025-04-15T20:36:13+08:00'}]}
{'text': '多云', 'temperature': '25'}<final_answer>深圳的天气是多云，温度为25摄氏度。</final_answer>

> Finished chain.


{'input': '深圳的天气怎么样？', 'output': '深圳的天气是多云，温度为25摄氏度。'}